In [ ]:
!pip uninstall -y tensorflow tf-nightly


In [ ]:
!pip install tensorflow==2.19


In [ ]:

# import libraries
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

print("TensorFlow version:", tf.__version__)  # should show 2.19.x



In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:


# Load TSV files
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Encode labels: ham=0, spam=1
train_df['label_num'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label_num'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Extract messages and labels
X_train = train_df['message'].values
y_train = train_df['label_num'].values
X_test = test_df['message'].values
y_test = test_df['label_num'].values

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")


In [ ]:
# Cell 4: Text vectorization and model

# Text vectorization
max_vocab = 10000
max_len = 100

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab,
    output_mode='int',
    output_sequence_length=max_len
)

# Adapt the vectorization layer on training data
vectorize_layer.adapt(X_train)

# Build the model
model = keras.Sequential([
    vectorize_layer,
    keras.layers.Embedding(input_dim=max_vocab, output_dim=16, mask_zero=True),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # output probability of "spam"
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


In [ ]:
# Cell 5: Function to predict message
def predict_message(pred_text):
    # Convert the text to a Tensor
    text_tensor = tf.convert_to_tensor([pred_text])

    # Get probability of "spam"
    prob = float(model.predict(text_tensor, verbose=0)[0][0])

    # Determine label
    label = "spam" if prob >= 0.5 else "ham"

    return [prob, label]

# Example usage
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)



In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
